In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
x = np.load("../sampling_data/random_sampling_x_85.npy",allow_pickle=True)
y = np.load("../sampling_data/random_sampling_y_85.npy", allow_pickle=True)
x_1, y_1 = x[0],y[0]
x_2, y_2 = x[1],y[1]
x_3, y_3 = x[2],y[2]
x_4, y_4 = x[3],y[3]
x_5, y_5 = x[4],y[4]

In [ ]:
resnet_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=X_1[0].shape,
    pooling=None
)
resnet_model.trainable = True

resnet_model.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [ ]:
for layer in resnet_model.layers[:-4]:
   layer.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(resnet_model.output)
x = tf.keras.layers.Flatten()(resnet_model.output)
# x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(resnet_model.input, x)

In [ ]:
# augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest')

train_generator = datagen.flow(x_5, y_5, batch_size=32, shuffle=True, seed=42)
valid_generator = datagen.flow(X_val, y_val, batch_size=32, seed=42)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val))
history = model.fit(train_generator,epochs=15, validation_data=valid_generator) # augmentation
results = model.evaluate(X_test, y_test)
print(results)

Epoch 1/15
139/139 [==============================] - 78s 460ms/step - loss: 3.6660 - accuracy: 0.8845 - val_loss: 0.6806 - val_accuracy: 0.6875
Epoch 2/15
139/139 [==============================] - 62s 444ms/step - loss: 0.1598 - accuracy: 0.9432 - val_loss: 0.6375 - val_accuracy: 0.6875
Epoch 3/15
139/139 [==============================] - 61s 437ms/step - loss: 0.1173 - accuracy: 0.9549 - val_loss: 0.3824 - val_accuracy: 0.8750
Epoch 4/15
139/139 [==============================] - 61s 434ms/step - loss: 0.1265 - accuracy: 0.9524 - val_loss: 1.1168 - val_accuracy: 0.5625
Epoch 5/15
139/139 [==============================] - 61s 434ms/step - loss: 0.1126 - accuracy: 0.9601 - val_loss: 1.0356 - val_accuracy: 0.7500
Epoch 6/15
139/139 [==============================] - 60s 433ms/step - loss: 0.1022 - accuracy: 0.9596 - val_loss: 0.3149 - val_accuracy: 0.8125
Epoch 7/15
139/139 [==============================] - 61s 437ms/step - loss: 0.1108 - accuracy: 0.9610 - val_loss: 0.8442 - val_ac

In [ ]:
model.save("./model/resnet50_model5.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.reshape(624)
y_pred1 = np.zeros(624)
for i in range(len(y_pred)):
  if y_pred[i] > 0.5:
    y_pred1[i]=1

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
target_names = ['NORMAL', 'PNEUMONIA']
print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1, target_names=target_names))

with open("./result/resnet50_result5.txt", "w") as text_file: 
  print(classification_report(y_test, y_pred1, target_names=target_names), file=text_file)

[[165  69]
 [  9 381]]
              precision    recall  f1-score   support

      NORMAL       0.95      0.71      0.81       234
   PNEUMONIA       0.85      0.98      0.91       390

    accuracy                           0.88       624
   macro avg       0.90      0.84      0.86       624
weighted avg       0.88      0.88      0.87       624

